In [1]:
%config IPCompleter.greedy=True

# import matplotlib.pyplot as plt
import numpy as np
import time
import zhinst.core
import laboneq
from laboneq.simple import *
import ctypes
import helpers
import matplotlib.pyplot as plt
from picosdk.ps5000a import ps5000a as ps
from picosdk.functions import adc2mV, assert_pico_ok, mV2adc
import picoscope_module as pm
#install_token("LabOneQ.AllQuantumInOne.")
DO_EMULATION = False # run in emulation mode by default

print(laboneq.__version__)

2.11.0


In [2]:
points_map={
    'level1': [-0.1, 0.1], 
    'level2': [0.1, -0.1],

 
}


times_map = {'pulse_time': 1e-3, #was 10
             'dead_time': 10e-3,
             'trigger_time': 1e-3}


START = -0.1 
STOP = 0.1
STEPS = 4 

## define length sweep parameter
level_sweep_parameter = LinearSweepParameter(
    uid="gate2_sweep", start=START, stop=STOP, count=STEPS,)


no_level_sweep_parameter_1 = LinearSweepParameter(
    uid="gate2_sweep", start=START, stop=START, count=STEPS,)


no_level_sweep_parameter_2 = LinearSweepParameter(
    uid="gate2_sweep", start=STOP, stop=STOP, count=STEPS,)

In [3]:
timebase = 628
preTriggerSamples = 10
pico_sampling_rate = (timebase - 3) / 62500000
postTriggerSamples = int(np.ceil(20e-3/pico_sampling_rate)) #Measurment length was 8
TriggerLevel=1

maxSamples = preTriggerSamples + postTriggerSamples
ready = ctypes.c_int16(0)
check = ctypes.c_int16(0)
maxADC = ctypes.c_int16()

chandle, status = pm.initialize_picoscope()
chARange, chBRange, status = pm.setup_channels(chandle)
# find maximum ADC count value

pm.setup_trigger(chandle, chARange, chBRange, maxADC,TriggerLevel)


{'maximumValue': 0,
 'setTriggerChannelPropertiesV2': 0,
 'setTriggerChannelConditionsV2': 0,
 'setTriggerChannelDirections': 0}

In [4]:
descriptor="""
instrument_list:
  HDAWG:
  - address: DEV8721
    uid: device_hdawg
    interface: usb
connections:
  device_hdawg:
    - rf_signal: q0/fg4_line
      ports: [SIGOUTS/0]
    - rf_signal: q0/fg6_line
      ports: [SIGOUTS/1]
"""
#Change back to to SIGOUTS/2 and SIGOUTS/3
#And change fg8 and fg9
device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="127.0.0.1",
    server_port="8004",
    setup_name="ZI_HDAWG",
)

c:\Users\B15measure2\anaconda3\envs\laboneq\lib\site-packages\laboneq\dsl\device\_device_setup_generator.py:1104: FutureWarning: 'instrument_list' section is deprecated in setup descriptor, use 'instruments' instead.
  warnings.warn(


In [5]:
# define pulse shapes
@pulse_library.register_pulse_functional
def ramp(x,start=0,stop=1 , **_):
    pulse=start+ (stop-start)*(x+1)/2 # Time normalized to (-1,1)
    return pulse



compress_level_pulse_level1_gate1=pulse_library.const(uid="compress_levell1g1",length=times_map['pulse_time'],amplitude=1,can_compress=True)
compress_level_pulse_level1_gate2=pulse_library.const(uid="compress_levell1g2",length=times_map['pulse_time'],amplitude=1,can_compress=True)

compress_level_pulse_level2_gate1 = pulse_library.const(uid="step_level_g1",length=times_map['pulse_time'],amplitude=1,can_compress=True)
compress_level_pulse_level2_gate2 = pulse_library.const(uid="step_level_g2",length=times_map['pulse_time'],amplitude=1,can_compress=True)






In [6]:
## Create Experiment
exp = Experiment(
    "Singlet-triplet qubit example",
    signals=[
        ExperimentSignal("gate1"),
        ExperimentSignal("gate2"),
    ],
)






    # define experiment
NUM_REP = 2

with exp.acquire_loop_rt(
    uid=("shots"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid='sweep', 
    parameter=level_sweep_parameter,alignment=SectionAlignment.LEFT ):
        exp.play(signal="gate1", pulse=compress_level_pulse_level1_gate1, amplitude=no_level_sweep_parameter_1)
        exp.play(signal="gate2", pulse=compress_level_pulse_level1_gate2, amplitude=no_level_sweep_parameter_1) 
        exp.play(signal="gate1", pulse=compress_level_pulse_level2_gate1, amplitude = level_sweep_parameter)
        exp.play(signal="gate2", pulse=compress_level_pulse_level2_gate2, amplitude = level_sweep_parameter)

"""         with exp.section(
            uid=("level2"),
            length=times_map['pulse_time'], 
            alignment=SectionAlignment.LEFT,
        ):

            exp.play(signal="gate1", pulse=compress_level_pulse_level2_gate1, amplitude = level_sweep_parameter)
            exp.play(signal="gate2", pulse=compress_level_pulse_level2_gate2, amplitude = level_sweep_parameter)
        with exp.section(
            uid=("level3"),
            length=times_map['pulse_time'], 
            alignment=SectionAlignment.LEFT,
        ):
            exp.play(signal="gate1", pulse=compress_level_pulse_level3_gate1)
            exp.play(signal="gate2", pulse=compress_level_pulse_level3_gate2)     """

'         with exp.section(\n            uid=("level2"),\n            length=times_map[\'pulse_time\'], \n            alignment=SectionAlignment.LEFT,\n        ):\n\n            exp.play(signal="gate1", pulse=compress_level_pulse_level2_gate1, amplitude = level_sweep_parameter)\n            exp.play(signal="gate2", pulse=compress_level_pulse_level2_gate2, amplitude = level_sweep_parameter)\n        with exp.section(\n            uid=("level3"),\n            length=times_map[\'pulse_time\'], \n            alignment=SectionAlignment.LEFT,\n        ):\n            exp.play(signal="gate1", pulse=compress_level_pulse_level3_gate1)\n            exp.play(signal="gate2", pulse=compress_level_pulse_level3_gate2)     '

In [7]:
# shortcut to the logical signal group q0
lsg = device_setup.logical_signal_groups["q0"].logical_signals

# define signal map
map_signals = {
    "gate1" : lsg["fg4_line"],
    "gate2" : lsg["fg6_line"]
}

In [8]:
source = ps.PS5000A_CHANNEL["PS5000A_CHANNEL_B"] #set the buffer
noOfCaptures = NUM_REP
status,timeIntervalns = pm.capture_rapid_data_block(chandle, preTriggerSamples, postTriggerSamples, timebase, maxSamples,noOfCaptures)


# # create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)
# set experiment calibration and signal map
exp.set_signal_map(map_signals)



if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)
print("Loaded exp")

session.run(exp)
print("Running exp finished")
# Check for data collection to finish using ps5000aIsReady
while ready.value == check.value:
    status["isReady"] = ps.ps5000aIsReady(chandle, ctypes.byref(ready))

buffersMax, buffersMin = pm.create_rapid_buffer(chandle, source, maxSamples, noOfCaptures)
pm.getValuesRapid(chandle,maxSamples,noOfCaptures)

2023.08.08 18:44:44.162 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\B15measure2\HDAWG\laboneq_output\log
2023.08.08 18:44:44.165 laboneq.controller.controller  INFO   VERSION: laboneq 2.11.0
2023.08.08 18:44:44.166 laboneq.controller.devices.device_collection INFO   Connecting to data server at 127.0.0.1:8004


2023.08.08 18:44:44.499 laboneq.controller.communication INFO   Connected to Zurich Instruments LabOne Data Server version 23.02 at 127.0.0.1:8004
2023.08.08 18:44:44.782 laboneq.controller.devices.device_collection INFO   Configuring the device setup
2023.08.08 18:44:44.982 laboneq.controller.devices.device_collection INFO   The device setup is configured
Loaded exp
2023.08.08 18:44:45.227 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.08.08 18:44:45.479 laboneq.compiler.scheduler.scheduler INFO   Schedule completed
2023.08.08 18:44:50.962 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 59
2023.08.08 18:44:50.965 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 256
2023.08.08 18:44:50.966 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.08.08 18:44:51.183 laboneq.controller.controller  INFO   Starting near-time execution...
2023.08.08 18:44:51.577 laboneq.controller.controller

In [9]:
# convert ADC counts data to mV
cmaxSamples = ctypes.c_int32(maxSamples)

# Create time data
time_stamp=int(time.time())
time_array = np.linspace(0, (cmaxSamples.value - 1) * timeIntervalns.value, cmaxSamples.value)
with open('Data/data_%s.npy'%(time_stamp), 'wb') as f:
    np.save(f,time_array)
    for i in range(noOfCaptures):
        np.save(f,adc2mV(buffersMax[i], chARange, maxADC))


with open('Data/data_%s.txt'%(time_stamp), 'w') as f:
    f.write(str(points_map)+'\n')
    f.write(str(times_map)+'\n')
    #f.write(str(exp)+'\n')


# display status returns
# print(status)

# Stop the scope
status["stop"] = ps.ps5000aStop(chandle)
assert_pico_ok(status["stop"])


 # Close unit Disconnect the scope 
status["close"]=ps.ps5000aCloseUnit(chandle)
assert_pico_ok(status["close"])